In [20]:
%pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate
%pip install numpy==1.26.4 scipy==1.11.4 pandas==2.2.2 lxml==5.2.2 scikit-learn==1.5.0 joblib==1.4.2 statsmodels==0.14.2 lifelines==0.28.0 pygam==0.9.1 l0bnb==1.0.0 torch==2.3.0 torchvision==0.18.0 pytorch-lightning==2.2.5 torchinfo==1.8.0 torchmetrics==1.4.0.post0 ISLP==0.4.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
created virtual environment CPython3.13.1.final.0-64 in 3714ms
  creator CPython3Windows(dest=C:\Users\rding\Documents\GitHub\Lumber-Project\myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=C:\Users\rding\AppData\Local\pypa\virtualenv)
    added seed packages: pip==25.0.1
  activators BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


'source' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + c:\Python313\python.exe C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4fad\vendored-meson\meson\meson.py setup C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4fad C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4fad\.mesonpy-1hziy9tc -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4fad\.mesonpy-1hziy9tc\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4fad
      Build dir: C:\Users\rding\AppData\Local\Temp\pip-install-vjautdf7\numpy_7c05a46dfa454d21bae63ba41d9d4f

In [8]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay  # Business Day offset

# Load datasets
delivery_df = pd.read_csv('DELIVERY TRACKDB.csv')
dff_df = pd.read_csv('DFF.csv')
gas_df = pd.read_csv('GASREGW.csv')
lumber_df = pd.read_csv('WPU081.csv')

# Standardize date formatting
delivery_df['DATE'] = pd.to_datetime(delivery_df['DATE'])

# Print column names to verify what pandas sees
print(delivery_df.columns.tolist())



['DATE', 'EXPECTED TYPE A TRUCKS', 'ACTUAL  TYPE A TRUCKS', 'OUTSIDE EXTRA  TYPE A TRUCKS', 'EXPECTED  TYPE B FORK TRUCKS', 'ACTUAL  TYPE B TRUCKS', 'EXPECTED  TYPE C TRUCKS', 'ACTUAL  TYPE C TRUCKS', 'LARGE DELIVERIES \n(LBS)', 'SMALL DELIVERIES (LBS)', 'LARGE DELIVERIES \n(QTY)', 'SMALL DELIVERIES \n(QTY)', ' RESCHEDULES\n(QTY)', 'RESCHEDULES\n(LBS)']


C:\Users\rding\AppData\Local\Temp\ipykernel_23468\2139381472.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  delivery_df['DATE'] = pd.to_datetime(delivery_df['DATE'])


In [9]:
# Create a clean mapping based on the actual column names
column_mapping = {
    'DATE': 'DATE',
    'EXPECTED TYPE A TRUCKS': 'EXPECTED TYPE A TRUCKS',
    'ACTUAL  TYPE A TRUCKS': 'ACTUAL TYPE A TRUCKS',
    'OUTSIDE EXTRA  TYPE A TRUCKS': 'OUTSIDE EXTRA TYPE A TRUCKS',
    'EXPECTED  TYPE B FORK TRUCKS': 'EXPECTED TYPE B FORK TRUCKS',
    'ACTUAL  TYPE B TRUCKS': 'ACTUAL TYPE B TRUCKS',
    'EXPECTED  TYPE C TRUCKS': 'EXPECTED TYPE C TRUCKS',
    'ACTUAL  TYPE C TRUCKS': 'ACTUAL TYPE C TRUCKS',
    'LARGE DELIVERIES \n(LBS)': 'LARGE DELIVERIES (LBS)',
    'SMALL DELIVERIES (LBS)': 'SMALL DELIVERIES (LBS)',
    'LARGE DELIVERIES \n(QTY)': 'LARGE DELIVERIES (QTY)',
    'SMALL DELIVERIES \n(QTY)': 'SMALL DELIVERIES (QTY)',
    ' RESCHEDULES\n(QTY)': 'RESCHEDULES (QTY)',
    'RESCHEDULES\n(LBS)': 'RESCHEDULES (LBS)'
}

# Rename columns
delivery_df = delivery_df.rename(columns=column_mapping)

# Now convert string weight columns to numeric
delivery_df['LARGE DELIVERIES (LBS)'] = pd.to_numeric(delivery_df['LARGE DELIVERIES (LBS)'].astype(str).str.replace(',', ''), errors='coerce')
delivery_df['SMALL DELIVERIES (LBS)'] = pd.to_numeric(delivery_df['SMALL DELIVERIES (LBS)'].astype(str).str.replace(',', ''), errors='coerce')
delivery_df['RESCHEDULES (LBS)'] = pd.to_numeric(delivery_df['RESCHEDULES (LBS)'].astype(str).str.replace(',', ''), errors='coerce')

In [10]:
# Now proceed with the rest of the preprocessing
dff_df['observation_date'] = pd.to_datetime(dff_df['observation_date'])
gas_df['observation_date'] = pd.to_datetime(gas_df['observation_date'])
lumber_df['observation_date'] = pd.to_datetime(lumber_df['observation_date'])

# Create a business day date range based on delivery data
b_day_range = pd.DatetimeIndex(delivery_df['DATE'].unique()).sort_values()

# Set up economic indicators dataframes with proper date indices
dff_df = dff_df.set_index('observation_date')
gas_df = gas_df.set_index('observation_date')
lumber_df = lumber_df.set_index('observation_date')

# Function to assign the most recent value to each business day
def align_to_business_days(df, business_days):
    # Create a temporary dataframe with all calendar days
    full_range = pd.date_range(start=min(business_days), end=max(business_days))
    temp_df = df.reindex(full_range, method='ffill')
    
    # Extract only the business days we need
    return temp_df.reindex(business_days)

# Align economic indicators to business days in delivery data
dff_aligned = align_to_business_days(dff_df, b_day_range)
gas_aligned = align_to_business_days(gas_df, b_day_range)
lumber_aligned = align_to_business_days(lumber_df, b_day_range)

# Create economic indicators dataframe with business days only
economic_indicators = pd.DataFrame(index=b_day_range)
economic_indicators['DFF'] = dff_aligned['DFF']
economic_indicators['GASREGW'] = gas_aligned['GASREGW']
economic_indicators['WPU081'] = lumber_aligned['WPU081']
economic_indicators = economic_indicators.reset_index().rename(columns={'index': 'DATE'})

# Final merge with delivery data
merged_df = pd.merge(delivery_df, economic_indicators, on='DATE', how='left')

In [13]:
merged_df.head(20)

,DATE,EXPECTED TYPE A TRUCKS,ACTUAL TYPE A TRUCKS,OUTSIDE EXTRA TYPE A TRUCKS,EXPECTED TYPE B FORK TRUCKS,ACTUAL TYPE B TRUCKS,EXPECTED TYPE C TRUCKS,ACTUAL TYPE C TRUCKS,LARGE DELIVERIES (LBS),SMALL DELIVERIES (LBS),LARGE DELIVERIES (QTY),SMALL DELIVERIES (QTY),RESCHEDULES (QTY),RESCHEDULES (LBS),DFF,GASREGW,WPU081
0,2022-01-03,36,21,12,12,6,12,15,19002960,923670,518,294,5,6620,0.08,3.281,404.213
1,2022-01-04,36,21,6,12,9,12,15,16356930,1153725,476,371,6,6771,0.08,3.281,404.213
2,2022-01-05,36,24,3,12,9,12,15,12386130,1147290,406,280,7,16151,0.08,3.281,404.213
3,2022-01-06,36,18,3,12,6,12,15,16140180,1025670,483,357,8,10504,0.08,3.281,404.213
4,2022-01-07,36,21,9,12,9,12,15,20495640,770505,602,280,6,14142,0.08,3.281,404.213
5,2022-01-10,36,21,6,12,9,12,9,18340125,1188240,504,364,7,20061,0.08,3.295,404.213
6,2022-01-11,36,18,9,12,9,12,12,19898250,1132980,616,427,6,10076,0.08,3.295,404.213
7,2022-01-12,36,18,3,12,9,12,15,14550255,1015365,399,413,4,9623,0.08,3.295,404.213
8,2022-01-13,36,21,12,12,9,12,15,20526750,1319250,525,420,7,364,0.08,3.295,404.213
9,2022-01-14,36,21,12,12,9,12,15,15294225,1232520,301,560,5,7204,0.08,3.295,404.213


In [14]:
# Create all derived metrics from the data dictionary
merged_df['Reschedule_Rate'] = merged_df['RESCHEDULES (QTY)'] / (merged_df['LARGE DELIVERIES (QTY)'] + merged_df['SMALL DELIVERIES (QTY)'])

merged_df['Weight_Adjusted_Reschedule_Impact'] = merged_df['RESCHEDULES (LBS)'] / (merged_df['LARGE DELIVERIES (LBS)'] + merged_df['SMALL DELIVERIES (LBS)'])

merged_df['Truck_Utilization_Efficiency'] = (merged_df['ACTUAL TYPE A TRUCKS'] + merged_df['ACTUAL TYPE B TRUCKS'] + merged_df['ACTUAL TYPE C TRUCKS']) / (merged_df['EXPECTED TYPE A TRUCKS'] + merged_df['EXPECTED TYPE B FORK TRUCKS'] + merged_df['EXPECTED TYPE C TRUCKS'])

# This is the one I missed earlier
merged_df['Delivery_Fulfillment_Rate'] = 1 - merged_df['Reschedule_Rate']

merged_df['Type_A_Truck_Accuracy'] = merged_df['ACTUAL TYPE A TRUCKS'] / merged_df['EXPECTED TYPE A TRUCKS']
merged_df['Type_B_Truck_Accuracy'] = merged_df['ACTUAL TYPE B TRUCKS'] / merged_df['EXPECTED TYPE B FORK TRUCKS']
merged_df['Type_C_Truck_Accuracy'] = merged_df['ACTUAL TYPE C TRUCKS'] / merged_df['EXPECTED TYPE C TRUCKS']

merged_df['Average_Delivery_Size'] = (merged_df['LARGE DELIVERIES (LBS)'] + merged_df['SMALL DELIVERIES (LBS)']) / (merged_df['LARGE DELIVERIES (QTY)'] + merged_df['SMALL DELIVERIES (QTY)'])

# Handle potential infinities and NaNs
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
merged_df = merged_df.fillna(method='ffill')  # Forward fill for time series data
merged_df = merged_df.fillna(merged_df.mean())  # Fill remaining NaNs with column means

C:\Users\rding\AppData\Local\Temp\ipykernel_23468\3782671942.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df = merged_df.fillna(method='ffill')  # Forward fill for time series data


In [15]:
from statsmodels.tsa.seasonal import seasonal_decompose

# For seasonal decomposition, we need to set the date as index
ts_df = merged_df.set_index('DATE')

# For business day data, typical periods would be:
# 5 for weekly patterns (5 business days per week)
# 21 for monthly patterns (approx. 21 business days per month)

# Perform decomposition on Reschedule_Rate (or other metric of interest)
result_reschedule = seasonal_decompose(ts_df['Reschedule_Rate'].dropna(), model='additive', period=21)

# Add components back to the dataframe
ts_df['Trend_Reschedule_Rate'] = result_reschedule.trend
ts_df['Seasonal_Reschedule_Rate'] = result_reschedule.seasonal
ts_df['Residual_Reschedule_Rate'] = result_reschedule.resid

# Perform decomposition on Truck_Utilization_Efficiency
result_truck = seasonal_decompose(ts_df['Truck_Utilization_Efficiency'].dropna(), model='additive', period=21)

# Add components back to the dataframe
ts_df['Trend_Truck_Utilization'] = result_truck.trend
ts_df['Seasonal_Truck_Utilization'] = result_truck.seasonal
ts_df['Residual_Truck_Utilization'] = result_truck.resid

# Reset index for continued analysis
merged_df = ts_df.reset_index()

In [16]:
merged_df.to_csv('lumber_delivery_preprocessed.csv', index=False)

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   DATE                               763 non-null    datetime64[ns]
 1   EXPECTED TYPE A TRUCKS             763 non-null    int64         
 2   ACTUAL TYPE A TRUCKS               763 non-null    int64         
 3   OUTSIDE EXTRA TYPE A TRUCKS        763 non-null    int64         
 4   EXPECTED TYPE B FORK TRUCKS        763 non-null    int64         
 5   ACTUAL TYPE B TRUCKS               763 non-null    int64         
 6   EXPECTED TYPE C TRUCKS             763 non-null    int64         
 7   ACTUAL TYPE C TRUCKS               763 non-null    int64         
 8   LARGE DELIVERIES (LBS)             763 non-null    int64         
 9   SMALL DELIVERIES (LBS)             763 non-null    int64         
 10  LARGE DELIVERIES (QTY)             763